In [159]:
import scanpy as sc
import os
import anndata
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from scanpy.metrics.specificity.plot import marker_genes_distribution, one_v_max_genelist
import numpy as np
from scanpy.metrics.specificity.get_data import get_average_celltype_counts,get_spe
from scanpy.metrics.specificity.compute import one_v_max_matrix
from joblib import dump, load
import pickle
import seaborn as sns
import copy
os.chdir('C:\\Users\\ipmc\\Documents\\Metrics_results')

## Load data

In [34]:
DATA_PATH = r'data/'
MODEL_PATH = r'models/'

In [35]:
barbry_file = 'HCA_Barbry_Grch38_Raw_filter_Norm.h5ad'
meyer_file = 'lung.cellxgene.h5ad'

In [36]:
barbry = anndata.read_h5ad(DATA_PATH + 'HCA_Barbry_Grch38_Raw_filter_Norm.h5ad')
meyer = anndata.read_h5ad(DATA_PATH + 'lung.cellxgene.h5ad')

C:\Users\ipmc\.conda\envs\scanpypip\lib\site-packages\anndata\compat\__init__.py:158: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
C:\Users\ipmc\.conda\envs\scanpypip\lib\site-packages\anndata\compat\__init__.py:158: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [148]:
a=np.array(barbry.X.todense()[:,0])


In [136]:
barbry.obs

,batch,donor,manip,method,position,CellType,Sex,Age
D367_Brus_Nas1_CCATGTCTCTAAGCCA-1-27,0,D367,D367_Brus_Nas1,Brushing,Nasal,Multiciliated N,male,27.0
D322_Biop_Nas1_CGTCCATCAAATCCGT-1-0,0,D322,D322_Biop_Nas1,Biopsy,Nasal,Suprabasal N,female,30.0
D363_Brus_Nas1_GGCAATTTCGATCCCT-1-23,0,D363,D363_Brus_Nas1,Brushing,Nasal,Secretory N,female,24.0
D353_Brus_Nas1_CCAGCGAGTACACCGC-1-16,0,D353,D353_Brus_Nas1,Brushing,Nasal,Multiciliated N,female,30.0
D367_Brus_Nas1_AGGTCCGTCCTGCTTG-1-27,0,D367,D367_Brus_Nas1,Brushing,Nasal,Secretory N,male,27.0
...,...,...,...,...,...,...,...,...
D367_Biop_Pro1_TCGTAGAGTTTGGGCC-1-28,0,D367,D367_Biop_Pro1,Biopsy,Proximal,Basal,male,27.0
D353_Brus_Nas1_CGGACACTCACCTTAT-1-16,0,D353,D353_Brus_Nas1,Brushing,Nasal,Multiciliated N,female,30.0
D367_Biop_Pro1_GTGCATACAGGTCTCG-1-28,0,D367,D367_Biop_Pro1,Biopsy,Proximal,Basal,male,27.0
D372_Biop_Int2_GTACTTTTCTTGTACT-1-34,0,D372,D372_Biop_Int2,Biopsy,Intermediate,Suprabasal,female,35.0


## Détermination des gènes marqueurs selon plusieurs méthodes

On va tester la plupart des méthodes de ranking_genes_group.
Pour ce qui est de la spécificité, on peut en faire plusieurs cat. Dans un premier temps, on garde les high et les unique et on regarde ce que ça donne (combien yen a etc...). On pourra ensuite jeter un coup d'oeil aux multi spé mais yaura surement besoin de la métrique magique.

In [106]:
sc.tl.rank_genes_groups(barbry, 'CellType', method='t-test')
#sc.pl.rank_genes_groups(barbry, n_genes=20, sharey=False)

ttest_de =  copy.deepcopy(barbry.uns['rank_genes_groups'])

In [107]:
sc.tl.rank_genes_groups(barbry, 'CellType', method='wilcoxon')
#sc.pl.rank_genes_groups(barbry, n_genes=20, sharey=False)

wilco_de = copy.deepcopy(barbry.uns['rank_genes_groups'])

In [110]:
#sc.tl.rank_genes_groups(barbry, 'CellType', method='logreg')
#sc.pl.rank_genes_groups(barbry, n_genes=20, sharey=False)

#logreg_de = copy.deepcopy(barbry.uns['rank_genes_groups'])

C:\Users\ipmc\.conda\envs\scanpypip\lib\site-packages\anndata\_core\anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])
C:\Users\ipmc\.conda\envs\scanpypip\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [135]:
barbry.uns['rank_genes_groups']['names'].dtype

dtype((numpy.record, [('AT1', 'O'), ('AT2', 'O'), ('B cells', 'O'), ('Basal', 'O'), ('Brush cells', 'O'), ('Cycling Basal', 'O'), ('Dendritic', 'O'), ('Deuterosomal', 'O'), ('Endothelial', 'O'), ('Fibroblast', 'O'), ('Ionocyte', 'O'), ('LT/NK', 'O'), ('Macrophage', 'O'), ('Mast cells', 'O'), ('Monocyte', 'O'), ('Multiciliated', 'O'), ('Multiciliated N', 'O'), ('PNEC', 'O'), ('Pericyte', 'O'), ('Plasma cells', 'O'), ('Precursor', 'O'), ('SMG Goblet', 'O'), ('Secretory', 'O'), ('Secretory N', 'O'), ('Serous', 'O'), ('Smooth muscle', 'O'), ('Suprabasal', 'O'), ('Suprabasal N', 'O')]))

In [99]:
dic['d']

'nounou'

In [145]:
def dict_to_df(DE_dico):
    celltypes = barbry.obs['CellType'].cat.categories
    df = pd.DataFrame(index = np.linspace(1,20,20)  , columns = celltypes )
    for celltype in celltypes:
        df[celltype] = DE_dico['names'][celltype][0:20]
    return df

## Load specificity

In [157]:
def find_max_celltype(gene,adata,partition_key="CellType"):
    get_average_celltype_counts(adata , partition_key = partition_key)
    gene_index = int(np.where(adata.var.index == gene)[0])
    max_celltype_index = np.argmax(adata.varm['ave_celltype_counts_' + partition_key][gene_index,:])
    max_celltype = list(adata.obs[partition_key].cat.categories)[max_celltype_index]
    return(max_celltype)

def full_spe_matrix(adata, partition_key: str = 'CellType') :
    shannon = get_spe(adata, spe_metric='shannon', partition_key=partition_key)
    tau = get_spe(adata, spe_metric='tau', partition_key=partition_key)
    gini = get_spe(adata, spe_metric='gini', partition_key=partition_key)
    one_v_max = one_v_max_matrix(adata,partition_key=partition_key)
    spe_matrix = pd.DataFrame(columns=['gene',
                                       'most expressed celltype',
                                       'one_v_max',
                                       'shannon',
                                       'tau',
                                       'gini'])
    for gene in adata.var.index :
        print()
        spe_matrix.loc[gene,:] = [gene,
                                  find_max_celltype(gene,adata,partition_key=partition_key),
                                  max(one_v_max[gene]), 
                                  shannon[gene],
                                  tau[gene],
                                  gini[gene]]
    return spe_matrix

In [ ]:
spe_matrix = full_spe_matrix(barbry, partition_key='CellType')
clf_Deprez_366obs = load(MODEL_PATH + 'clf_train_Deprez_366obs.joblib')
spe_type = clf_Deprez_366obs.predict(spe_matrix[['one_v_max','shannon','tau','gini']])
spec_dict={'0':'equirep','1':'low','2':'multi-spec','3':'high','4':'unique'}
spe_matrix['spe_type'] = pd.Series(spe_type, index = spe_matrix.index).apply(str).replace(spec_dict)

In [169]:
spe_matrix

,gene,most expressed celltype,one_v_max,shannon,tau,gini,spe_type
AL669831.5,AL669831.5,AT2,1.56192,0.637306,0.742319,0.391246,low
LINC00115,LINC00115,PNEC,1.22682,0.628045,0.626468,0.399453,equirep
FAM41C,FAM41C,Deuterosomal,1.12528,0.637185,0.663437,0.393725,equirep
NOC2L,NOC2L,Precursor,1.46304,0.673486,0.569871,0.219136,equirep
PLEKHN1,PLEKHN1,Suprabasal N,1.83215,0.519458,0.867355,0.66531,low
...,...,...,...,...,...,...,...
AC007325.4,AC007325.4,Deuterosomal,3.16325,0.538201,0.907686,0.615957,high
AC007325.2,AC007325.2,Cycling Basal,1.07778,0.414222,0.886558,0.792062,multi-spec
AL354822.1,AL354822.1,Ionocyte,1.26802,0.604515,0.729433,0.467284,equirep
AC004556.1,AC004556.1,Precursor,1.42064,0.578837,0.840537,0.550767,multi-spec
